In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession,SQLContext
import numpy as np
import pandas as pd
from pyspark.sql.types import *

spark = SparkSession.builder.appName('readJson').getOrCreate()



In [2]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("field2", StringType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("job_id", IntegerType(), True)])

In [3]:
# Read JSONs
#df = spark.read("hired1.csv")
#df = spark.read.format("csv").option("header","false","schema"=schema).load("hired1.csv")
df = spark.read.csv("hired1.csv",header=False,schema=schema)

In [28]:
df.show(1000)

+---+--------------------+--------------------+-------------+------+
| id|                name|              field2|department_id|job_id|
+---+--------------------+--------------------+-------------+------+
|  1|         Harold Vogt|2021-11-07T02:48:42Z|            2|    96|
|  2|            Ty Hofer|2021-05-30T05:43:46Z|            8|  null|
|  3|         Lyman Hadye|2021-09-01T23:27:38Z|            5|    52|
|  4|       Lotti Crowthe|2021-10-01T13:04:21Z|           12|    71|
|  5|      Gretna Lording|2021-10-10T22:22:17Z|            6|    80|
|  6|      Marlow Antecki|2021-04-23T23:45:42Z|            6|    95|
|  7|        Joan Rillett|2021-10-10T01:33:31Z|            9|    78|
|  8|    Ulrick Nucciotti|2021-07-24T01:28:40Z|            8|   169|
|  9|  Lucretia Northcote|2021-04-01T21:22:47Z|            9|     8|
| 10|        Arty Giacobo|2022-02-08T12:27:07Z|            6|    62|
| 11|        Libbi Dowtry|2021-07-05T04:55:10Z|            6|    41|
| 12|        Jacky Oldred|2021-06-

In [6]:
df.columns

['id', 'name', 'field2', 'department_id', 'job_id']

In [7]:
path_dept = './data/departments.csv'
path_jobs = './data/jobs.csv'
path_hiredemp = 'hired1.csv'


#df = pd.read_csv(path_dept,header=None, names=['id', 'tdepartment'])
#print(data_dept.iloc[0:3,])

df = pd.read_csv(path_hiredemp,header=None, names=['id', 'name', 'datefield', 'department_id', 'job_id'])

df.head()

,id,name,datefield,department_id,job_id
0,id,employee_name,hired_date,department_id,job_id
1,150,Valentia Kellegher,2021-11-13 13:30:304,51,NaN
2,151,Jarrett Oluwatoyin,2021-02-11 21:25:178,34,NaN
3,152,Kori Foxton,2021-04-10 14:02:385,95,NaN
4,153,Truman Garriock,2021-09-08 13:11:2611,88,NaN


In [8]:
df.columns

Index(['id', 'name', 'datefield', 'department_id', 'job_id'], dtype='object')

In [17]:
df.createOrReplaceTempView('hired')

In [18]:
spark.sql("""
SELECT * FROM hired  
  """).show()

+---+------------------+--------------------+---+----+
|_c0|               _c1|                 _c2|_c3| _c4|
+---+------------------+--------------------+---+----+
|  1|       Harold Vogt|2021-11-07T02:48:42Z|  2|  96|
|  2|          Ty Hofer|2021-05-30T05:43:46Z|  8|null|
|  3|       Lyman Hadye|2021-09-01T23:27:38Z|  5|  52|
|  4|     Lotti Crowthe|2021-10-01T13:04:21Z| 12|  71|
|  5|    Gretna Lording|2021-10-10T22:22:17Z|  6|  80|
|  6|    Marlow Antecki|2021-04-23T23:45:42Z|  6|  95|
|  7|      Joan Rillett|2021-10-10T01:33:31Z|  9|  78|
|  8|  Ulrick Nucciotti|2021-07-24T01:28:40Z|  8| 169|
|  9|Lucretia Northcote|2021-04-01T21:22:47Z|  9|   8|
| 10|      Arty Giacobo|2022-02-08T12:27:07Z|  6|  62|
| 11|      Libbi Dowtry|2021-07-05T04:55:10Z|  6|  41|
| 12|      Jacky Oldred|2021-06-20T11:59:41Z|  7|  29|
| 13|      Raine Mowett|2021-12-23T08:23:34Z|  8|  83|
| 14|   Melonie Slocomb|2021-12-30T23:08:59Z| 12| 121|
| 15|    Robers Swinden|2021-02-16T20:00:38Z|  5|  47|
| 16|     